# Face Detection

In [12]:
#import necessary modules

import numpy as np
import pandas as pd
import cv2
import os

## Load images from dir


In [4]:
#images path
img_dir = 'train_images'
# Path to the folder containing test images
test_img_dir = 'test_images'
# Create a mapping from folder names to integer labels
label_map = {'wmaathai': 1, 'others': 0}

: 

## Train FaceRecognizer Model

In [5]:
# initialize empty lists for training images and image-labels
faces = []
ids = []


: 

In [6]:
# Load Haar Cascade classifiers for frontal and side faces
front_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
side_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

: 

In [7]:
# Set to track processed face coordinates (to avoid duplicates)
processed_faces_coordinates = set()

: 

In [ ]:
# train FaceRecognizer model
for root, dirs, files in os.walk(img_dir):
    for filename in files:
    
        # Check if the file is an image (JPG, PNG, JPEG)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, filename)  # Get full image path relative to root
        
            # Read the image
            img = cv2.imread(image_path)

            # Skip invalid images
            if img is None:
                print(f"Failed to load image {image_path}")
                continue

            # Convert the image to grayscale (Haar cascades work with grayscale images)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Detect frontal faces
            front_faces = front_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            # Detect side faces
            side_faces = side_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            print(f"Detected {len(front_faces)} front face(s) and {len(side_faces)} side face(s) in {image_path}")

            #skip invalid images
            if img is None:
                print(f"Failed to load image {img_path}")
                continue

            # convert image to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # load Haar CascadeClasifier and detect faces
            # Load the pre-trained Haar cascade classifiers
            # Detect faces using both front and side face classifiers
            front_faces = front_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            side_faces = side_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            print(f"Detected {len(front_faces)} front face(s) and {len(side_faces)} side face(s) in {image_path}")

            # Process the front faces
            for (x, y, w, h) in front_faces:
                # Check if this face has already been processed
                if (x, y, w, h) in processed_faces_coordinates:
                    continue  # Skip if face already processed

                # Add the face coordinates to the set to avoid duplicate processing
                processed_faces_coordinates.add((x, y, w, h))

                # Extract face region from grayscale image
                face = gray_img[y:y + h, x:x + w]
                faces.append(face)  # Add the face to the list

                # Get the label from the folder name
                label = os.path.basename(root)  # Folder name as label
                if label in label_map:
                    ids.append(label_map[label])  # Add corresponding label

                # Save the cropped front face image
                face_img = img[y:y + h, x:x + w]  # Extract color face for saving
                face_filename = f"front_face_{os.path.basename(root)}_{filename}"
                cv2.imwrite(f'./processed_faces/{face_filename}', face_img)  # Save the front face image

            # Process the side faces
            for (x, y, w, h) in side_faces:
                # Check if this face has already been processed
                if (x, y, w, h) in processed_faces_coordinates:
                    continue  # Skip if face already processed

                # Add the face coordinates to the set to avoid duplicate processing
                processed_faces_coordinates.add((x, y, w, h))

                # Extract face region from grayscale image
                face = gray_img[y:y + h, x:x + w]
                faces.append(face)  # Add the face to the list

                # Get the label from the folder name
                label = os.path.basename(root)  # Folder name as label
                if label in label_map:
                    ids.append(label_map[label])  # Add corresponding label



Detected 1 front face(s) and 1 side face(s) in train_images\others\1.jpg
Detected 1 front face(s) and 1 side face(s) in train_images\others\1.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\11.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\11.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\12.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\12.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\13.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\13.jpg
Detected 0 front face(s) and 1 side face(s) in train_images\others\14.jpg
Detected 0 front face(s) and 1 side face(s) in train_images\others\14.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\15.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\15.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\16.jpg
Detected 1 front face(s) and 0 side face

: 

## Face Detection
-consider if a face is not detected

In [11]:
# Check if faces and ids are not empty before training
if len(faces) > 0 and len(ids) > 0:
    # Create and train the recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, np.array(ids))  # Train the recognizer with the faces and corresponding ids

    # Save the trained model
    recognizer.save('./trained/trained_model.yml')
    print("Model trained and saved successfully.")
else:
    print("No faces detected, model training skipped.")

AttributeError: module 'cv2' has no attribute 'face'

: 

In [50]:
# Train the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(ids))


: 

In [51]:
# Save the trained model
recognizer.save('/workspaces/faceid/trained/trained_model.yml')
print("Model trained and saved successfully.")


Model trained and saved successfully.


: 

# Testing

In [52]:
# Load the trained model for face recognition
recognizer.read('/workspaces/faceid/trained/trained_model.yml')

: 

In [55]:
# Loop through the test images
for test_image_name in os.listdir(test_img_dir):
    test_image_path = os.path.join(test_img_dir, test_image_name)

    test_img = cv2.imread(test_image_path)
    if test_img is None:
        print(f"Failed to load image {test_image_path}")
        continue  # Skip invalid images

    # Convert test image to grayscale for face detection
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the test image
    faces_detected = face_cascade.detectMultiScale(gray_img, 1.3, 5)

    # Variable to track if a recognized face is found
    is_wmaathai = False
    print(f"Processing image: {test_image_name}")

    # Process each face detected
    for (x, y, w, h) in faces_detected:
        face_region = gray_img[y:y + h, x:x + w]  # Extract the face region

       # Recognize the face using the trained recognizer
        label, confidence = recognizer.predict(face_region)

        # Log the label and confidence for debugging
        print(f"  Detected face with label: {label}, confidence: {confidence}")

        # check if value of recognition confidence is good
        if confidence < 100:  # Low confidence means a good match
            if label == 1:
                is_wmaathai = True
                print(f"{test_image_name} is wmaathai")
            else:
                print(f"{test_image_name} is not wmaathai")
        else:
            # If the confidence is too high, mark the face as unknown
            print(f"{test_image_name} is unknown (confidence too high)")

# If no faces were detected or recognized, output unknown
if len(faces_detected) == 0:
    print(f"No faces detected in {test_image_name}")

        

Processing image: 8.jpg
Processing image: 11.jpg
Processing image: 13.jpg
  Detected face with label: 0, confidence: 86.99078925398025
13.jpg is not wmaathai
Processing image: 9.jpg
Processing image: 16.jpg
  Detected face with label: 0, confidence: 107.71763313927563
16.jpg is unknown (confidence too high)
Processing image: 5.jpg
Processing image: 3.jpg
  Detected face with label: 0, confidence: 67.41206249413666
3.jpg is not wmaathai
Processing image: 14.jpg
  Detected face with label: 0, confidence: 103.99205966192766
14.jpg is unknown (confidence too high)
Processing image: 1.jpg
Processing image: 4.jpg
  Detected face with label: 0, confidence: 108.01684593306607
4.jpg is unknown (confidence too high)
Processing image: 15.jpg
Processing image: 20.jpg
Processing image: 7.jpg
Processing image: 2.jpg
Processing image: 12.jpg
Processing image: 10.jpg
  Detected face with label: 0, confidence: 91.24775460385793
10.jpg is not wmaathai
Processing image: 18.jpg
  Detected face with label:

: 

In [57]:
# Draw a rectangle around the face and display the label
        # color = (0, 255, 0)  # Green color for recognized faces    
#cv2.rectangle(test_img, (x, y), (x + w, y + h), color, 2)
            #cv2.putText(test_img, f'Person {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        #else:
            # If the confidence is high, mark the face as unknown
            #cv2.putText(test_img, 'Unknown', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Show the result
    #cv2.imshow(f"Recognized Faces - {test_image_name}", test_img)
    # Save the result to a file instead of showing it
    #cv2.imwrite("recognized_faces.jpg", test_img)

    #cv2.waitKey(0)  # Wait for any key to close the window

#cv2.destroyAllWindows()


: 